In [ ]:
import tensorflow
from  tensorflow.keras.applications.vgg16 import VGG16 as model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import sys
import os
import utils.const as const

In [ ]:
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

In [ ]:
directory=const.grpId

batchSize=30000 #proportional to ram

outputLen=7*7*512 # model dependent

failed=[]
success=[]

outputFile='vgg16.csv'

In [ ]:
alreadyDone={}
if os.path.isfile(outputFile):
  with open(outputFile,'r') as fp:
    for i in fp:
      arr=i.split(',')
      if len(arr) != outputLen+1:
        continue
      alreadyDone[arr[0]]=1

print(len(alreadyDone))

In [ ]:
listofFiles=[]
count=0
for root, subdirs, files in os.walk(directory):
  for fileName in files:
    if root.endswith('videos'):
      continue

    count+=1
    path=os.path.join(root,fileName)

    if path not in alreadyDone:
      listofFiles.append(path)

print(count,len(alreadyDone),len(listofFiles),len(alreadyDone)+len(listofFiles))

In [ ]:
print(listofFiles[:10])

In [ ]:
Model = model(weights='imagenet', include_top=False)

In [ ]:
def worker(low,high):
  try:
    vectors=[]
    for i in range(low,high):
      imagePath=listofFiles[i]

      image = load_img(imagePath, target_size=(224, 224))
      image = img_to_array(image)

      image = np.expand_dims(image, axis=0)
      image = preprocess_input(image)

      vector = Model.predict(image)
      vectors.append(vector.flatten())

    with open(outputFile,'a') as fp:
      for i in range(len(vectors)):
        success.append(listofFiles[i+low])
        fp.write(listofFiles[i+low])

        if vectors[i].size != outputLen:
          raise Exception('Excpected output length is not correct ',str(vectors[i].size),str(outputLen))

        for k in range(vectors[i].shape[0]):
          fp.write(','+str(vectors[i][k]))
        
        fp.write('\n')
  
  except Exception as e:
    print(e)
    for i in range(low,high):
      failed.append(listofFiles[i])

In [ ]:
iter=(int)(len(listofFiles)/batchSize)
if batchSize*iter!=len(listofFiles):
  iter+=1

print(iter)

In [ ]:
failed=[]
success=[]

for i in range(iter):
  print(i,iter)
  low,high=i*batchSize, min((i+1)*batchSize,len(listofFiles))
  worker(low,high)


In [ ]:
print(len(failed),len(success))